https://adventofcode.com/2020/day/14

In [1]:
import re
from itertools import product

In [2]:
datafile = 'data/14-1.txt'

In [3]:
with open(datafile) as fh:
    data = [y for y in (x.strip() for x in fh) if y]

In [4]:
len(data), data[:3]

(536,
 ['mask = 1X01XXX001101X00001100X1010X10101101',
  'mem[62085] = 231745',
  'mem[14249] = 1252796'])

In [5]:
def makemask(maskstr):
    return {
        'con': int(maskstr.replace('X', '1'), 2),
        'dis': int(maskstr.replace('X', '0'), 2)
    }
    

def applymask(mask, n):
    for k, v in mask.items():
        if k == 'dis':
            n = n | v
        elif k == 'con':
            n = n & v
        else:
            raise ValueError("Unknown mask type: %s %s" % (k, v))
    return n

In [6]:
n = int('10100101', 2)
print('n = ', n)
mask = makemask('X00X1001')
print('mask =', mask)
print('applied = ', applymask(mask, n))

n =  165
mask = {'con': 153, 'dis': 9}
applied =  137


In [7]:
def parse_data(line):
    if line.startswith('mask'):
        return ('mask', line[len('mask = '):])
    elif line.startswith('mem'):
        return ('mem', tuple(int(x) for x in re.findall(r'\d+', line)))
    else:
        raise ValueError('Unknown data: %s' % line)

In [8]:
[parse_data(line) for line in data[:3]]

[('mask', '1X01XXX001101X00001100X1010X10101101'),
 ('mem', (62085, 231745)),
 ('mem', (14249, 1252796))]

In [9]:
mask = None
registers = {}
for line in data:
    cmd, arg = parse_data(line)
    if cmd == 'mask':
        mask = makemask(arg)
    else:
        k, v = arg
        registers[k] = applymask(mask, v)

In [10]:
part_1 = sum(registers.values())
part_1

10050490168421

In [19]:
def make_floatmasks(maskstr):
    basemask = makemask(maskstr)
    del basemask['con']
    D = {
        'base': basemask,
        'masks': []
    }
    xis = [i for (i, x) in enumerate(maskstr) if x == 'X']
    for bits in product(['0', '1'], repeat=len(xis)):
        L = ['X'] * 36
        for i, b in zip(xis, bits):
            L[i] = b
        D['masks'].append(makemask(''.join(L)))
    return D

        
def apply_floatmasks(floatmasks, n):
    base = applymask(floatmasks['base'], n)
    masks = floatmasks['masks']
    if not masks:
        yield base
    for m in masks:
        yield applymask(m, base)

In [20]:
maskstr = '000000000000000000000000000000X1001X'
addr = 42
fm = make_floatmasks(maskstr)
list(apply_floatmasks(fm, addr))

[26, 27, 58, 59]

In [21]:
maskstr = '00000000000000000000000000000000X0XX'
addr = 26
fm = make_floatmasks(maskstr)
list(apply_floatmasks(fm, addr))

[16, 17, 18, 19, 24, 25, 26, 27]

In [22]:
floatmasks = None
registers = {}
for line in data:
    cmd, arg = parse_data(line)
    if cmd == 'mask':
        floatmasks = make_floatmasks(arg)
    else:
        k, v = arg
        for reg in apply_floatmasks(floatmasks, k):
            registers[reg] = v

In [23]:
part_2 = sum(registers.values())
part_2

2173858456958

## post

In [31]:
%%time

import re
from itertools import product

with open(datafile) as fh:
    data = [y for y in (x.strip() for x in fh) if y]

    
def parse_command(line):
    if line.startswith('mask'):
        return ('mask', line[len('mask = '):])
    elif line.startswith('mem'):
        return ('mem', tuple(int(x) for x in re.findall(r'\d+', line)))
    else:
        raise ValueError('Unknown command: %s' % line)

# Part 1
        
def make_mask(maskstr):
    return {
        'con': int(maskstr.replace('X', '1'), 2),
        'dis': int(maskstr.replace('X', '0'), 2)
    }
    

def apply_mask(mask, n):
    for k, v in mask.items():
        if k == 'dis':
            n = n | v
        elif k == 'con':
            n = n & v
        else:
            raise ValueError("Unknown mask type: %s %s" % (k, v))
    return n

mask = None
registers = {}
for line in data:
    cmd, arg = parse_command(line)
    if cmd == 'mask':
        mask = make_mask(arg)
    else:
        k, v = arg
        registers[k] = apply_mask(mask, v)

part_1 = sum(registers.values())

# Part 2

def make_floatmasks(maskstr):
    basemask = make_mask(maskstr)
    del basemask['con']
    D = {
        'base': basemask,
        'masks': []
    }
    xis = [i for (i, x) in enumerate(maskstr) if x == 'X']
    for bits in product(['0', '1'], repeat=len(xis)):
        L = ['X'] * 36
        for i, b in zip(xis, bits):
            L[i] = b
        D['masks'].append(make_mask(''.join(L)))
    return D

        
def apply_floatmasks(floatmasks, n):
    base = apply_mask(floatmasks['base'], n)
    masks = floatmasks['masks']
    if not masks:
        yield base
    for m in masks:
        yield apply_mask(m, base)


floatmasks = None
registers = {}
for line in data:
    cmd, arg = parse_command(line)
    if cmd == 'mask':
        floatmasks = make_floatmasks(arg)
    else:
        k, v = arg
        for reg in apply_floatmasks(floatmasks, k):
            registers[reg] = v

part_2 = sum(registers.values())

CPU times: user 91.5 ms, sys: 0 ns, total: 91.5 ms
Wall time: 90.4 ms


In [30]:
part_1, part_2

(10050490168421, 2173858456958)